In [1]:
import numpy as np
import torch.nn as nn
from PIL import Image
import glob
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import math

N = 6283
D_in = 400
D_in2 = 20
H = 100
D_out = 62

x_list = []
x_data = np.zeros([N, D_in])
x_data2 = np.zeros([N, 1, D_in2, D_in2])
y_data = np.zeros([N, D_out])
y_data2 = np.zeros([N])

print("a")
#Reading images into numpy array. All images are stored as a 20*20 array on x_data2.

for img_id in range(N + 1):
    for filename in glob.glob("C:\\Users\\ahmetcan\\Documents\\SWE\\SWE 582\\Google Street View\\trainResized\\" 
                              + str(img_id) + ".bmp"):
        im=Image.open(filename)
        pix = np.array(im)
        
        if len(pix.shape) > 2:
            pix2 = pix.mean(2)
            pix3 = pix2.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix2
        else:
            pix3 = pix.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix
            
        x_list.append(pix3)
        x_data[img_id - 1] = pix3

print("b")

a
b


In [2]:
#The labels of the images are converted to integers.

def conv_lbl_to_int(a):
    if a.islower():
        return ord(a) - 61
    elif a.isnumeric():
        return int(a)
    else:
        return ord(a) - 55
    
df_labels = pd.read_csv('trainLabels.csv',header=0)

for index, row in df_labels.iterrows():
    if index < N:
        y_data2[index] = conv_lbl_to_int(row['Class'])

y_data2

x_tensor2 = torch.Tensor(x_data2)

y_tensor2 = torch.Tensor(y_data2)
y_tensor2 = y_tensor2.type(torch.LongTensor)

print("x")
y_tensor2

x



 49
  8
 29
 ⋮ 
 25
 23
 27
[torch.LongTensor of size 6283]

In [3]:
#The model is establishing. The momentum and learning rate is determined after a few trials and errors.

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(80, 62)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.7)

batch_size = 600
numberOfBatches = math.ceil(N/batch_size)
idx_start = 0

total_epoch = 1000

loss_list = np.zeros(numberOfBatches * total_epoch)
plot_idx = 0

# Neural network, neural network with hidden layer, convolutional neural network without batches are tried before this model.

print("start")
for epoch in range(1, total_epoch):
    idx_start = 0
    for batch_idx in range(numberOfBatches):
        plot_idx = plot_idx + 1
        x_data3 = x_data2[idx_start:idx_start + batch_size,:,:,:]
        y_data3 = y_data2[idx_start:idx_start + batch_size]
        
        x_tensor3 = torch.Tensor(x_data3)
        
        y_tensor3 = torch.Tensor(y_data3)
        y_tensor3 = y_tensor3.type(torch.LongTensor)
        
        data, target = Variable(x_tensor3), Variable(y_tensor3)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        if epoch % 20 == 0:# and batch_idx == 5:
            print("epoch: ", epoch, "loss_b", loss)
        
        #loss_list[plot_idx] = loss.data.numpy()[0]
        optimizer.step()
        idx_start = idx_start + batch_size

start
epoch:  20 loss_b Variable containing:
 4.0749
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0739
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0830
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0899
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0828
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0796
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0826
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0853
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0750
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0756
[torch.FloatTensor of size 1]

epoch:  20 loss_b Variable containing:
 4.0766
[torch.FloatTensor of size 1]

epoch:  40 loss_b Variable containing:
 3.3295
[torch.FloatTensor of size 1]

epoch:  40 loss_b Variable containing:
 3.2670
[torch.Floa

epoch:  200 loss_b Variable containing:
 1.0696
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 1.0654
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 1.0939
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 1.2239
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 0.9866
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 0.5687
[torch.FloatTensor of size 1]

epoch:  220 loss_b Variable containing:
 1.0353
[torch.FloatTensor of size 1]

epoch:  220 loss_b Variable containing:
 1.0194
[torch.FloatTensor of size 1]

epoch:  220 loss_b Variable containing:
 0.9480
[torch.FloatTensor of size 1]

epoch:  220 loss_b Variable containing:
 1.1170
[torch.FloatTensor of size 1]

epoch:  220 loss_b Variable containing:
 1.0242
[torch.FloatTensor of size 1]

epoch:  220 loss_b Variable containing:
 1.0242
[torch.FloatTensor of size 1]

epoch:  220 loss_b Variable containing:
 0.9924
[tor

epoch:  400 loss_b Variable containing:
 0.6026
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.5998
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.5903
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.6703
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.5735
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.7570
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.7109
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.6484
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.7636
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.6450
[torch.FloatTensor of size 1]

epoch:  400 loss_b Variable containing:
 0.2895
[torch.FloatTensor of size 1]

epoch:  420 loss_b Variable containing:
 0.5556
[torch.FloatTensor of size 1]

epoch:  420 loss_b Variable containing:
 0.5793
[tor

epoch:  580 loss_b Variable containing:
 0.5965
[torch.FloatTensor of size 1]

epoch:  580 loss_b Variable containing:
 0.5183
[torch.FloatTensor of size 1]

epoch:  580 loss_b Variable containing:
 0.5907
[torch.FloatTensor of size 1]

epoch:  580 loss_b Variable containing:
 0.6246
[torch.FloatTensor of size 1]

epoch:  580 loss_b Variable containing:
 0.4682
[torch.FloatTensor of size 1]

epoch:  580 loss_b Variable containing:
 0.2087
[torch.FloatTensor of size 1]

epoch:  600 loss_b Variable containing:
 0.4191
[torch.FloatTensor of size 1]

epoch:  600 loss_b Variable containing:
 0.4211
[torch.FloatTensor of size 1]

epoch:  600 loss_b Variable containing:
 0.3927
[torch.FloatTensor of size 1]

epoch:  600 loss_b Variable containing:
 0.4989
[torch.FloatTensor of size 1]

epoch:  600 loss_b Variable containing:
 0.4325
[torch.FloatTensor of size 1]

epoch:  600 loss_b Variable containing:
 0.5727
[torch.FloatTensor of size 1]

epoch:  600 loss_b Variable containing:
 0.5092
[tor

epoch:  760 loss_b Variable containing:
 0.2894
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.6270
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.6171
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.5279
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.6672
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.5333
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.8509
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.6916
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.7135
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.7436
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.7267
[torch.FloatTensor of size 1]

epoch:  780 loss_b Variable containing:
 0.6013
[torch.FloatTensor of size 1]

epoch:  800 loss_b Variable containing:
 0.7970
[tor

epoch:  960 loss_b Variable containing:
 0.3938
[torch.FloatTensor of size 1]

epoch:  960 loss_b Variable containing:
 0.4717
[torch.FloatTensor of size 1]

epoch:  960 loss_b Variable containing:
 0.3743
[torch.FloatTensor of size 1]

epoch:  960 loss_b Variable containing:
 0.3972
[torch.FloatTensor of size 1]

epoch:  960 loss_b Variable containing:
 0.4929
[torch.FloatTensor of size 1]

epoch:  960 loss_b Variable containing:
 0.4909
[torch.FloatTensor of size 1]

epoch:  960 loss_b Variable containing:
 0.1777
[torch.FloatTensor of size 1]

epoch:  980 loss_b Variable containing:
 0.4560
[torch.FloatTensor of size 1]

epoch:  980 loss_b Variable containing:
 0.4748
[torch.FloatTensor of size 1]

epoch:  980 loss_b Variable containing:
 0.4109
[torch.FloatTensor of size 1]

epoch:  980 loss_b Variable containing:
 0.3958
[torch.FloatTensor of size 1]

epoch:  980 loss_b Variable containing:
 0.4227
[torch.FloatTensor of size 1]

epoch:  980 loss_b Variable containing:
 0.4109
[tor

In [7]:
#Error on training set is calculated

x_tensor2 = torch.Tensor(x_data2)

y_tensor2 = torch.Tensor(y_data2)
y_tensor2 = y_tensor2.type(torch.LongTensor)

data, target = Variable(x_tensor2), Variable(y_tensor2)

output    = model(data)
out_numpy = output.data.numpy()

success = 0

for i in range(N):
    if np.argmax(out_numpy[i]) == y_data2[i]:
        success = success + 1

print(success / N)

0.8222186853413974


In [5]:
#The test set is put on numpy array

N_test = 12503
x_test = np.zeros([N_test - N, 1, D_in2, D_in2])
for img_id in range(N, N_test + 1):
    for filename in glob.glob("C:\\Users\\ahmetcan\\Documents\\SWE\\SWE 582\\Google Street View\\testResized\\" 
                              + str(img_id) + ".bmp"):
        #print("-----")
        #print(img_id)
        #print(img_id - N - 1)
        im=Image.open(filename)
        pix = np.array(im)
        
        if len(pix.shape) > 2:
            pix2 = pix.mean(2)
            x_test[img_id - N - 1, 0, :, :] = pix2
        else:
            x_test[img_id - N - 1, 0, :, :] = pix
            
            
x_tensor_test = torch.Tensor(x_test)

data_test = Variable(x_tensor_test)

output_test    = model(data_test)
out_test_numpy = output_test.data.numpy()

print("out_test_numpy", out_test_numpy)
print("out_test_numpy.shape", out_test_numpy.shape)

out_test_numpy [[-39.66809082 -28.65432358 -46.37583923 ..., -49.77484512 -29.63959503
  -37.05094528]
 [-26.43435287 -18.48987007 -21.65607452 ..., -20.27994537 -25.2082119
  -23.90656853]
 [ -7.99835968  -4.91275167 -10.73672867 ..., -10.65221119 -14.58886147
   -5.79925299]
 ..., 
 [-12.41446972 -36.13616943 -17.34126854 ..., -45.06451797 -51.42379761
  -20.58192825]
 [-41.41284943 -35.85604477 -44.73301315 ..., -22.78478432 -17.01324272
  -39.40509796]
 [-62.81671524 -41.19134903 -53.05243683 ..., -24.21271324 -19.92098236
  -48.2232666 ]]
out_test_numpy.shape (6220, 62)


In [6]:
#The labels on test set are predicted to submit it on Kaggle.

def conv_int_to_lbl(a):
    if 36 <= a <= 61:#a.islower():
        return chr(int(a) + 61)
    elif 0 <= a <= 9:#elif a.isnumeric():
        #return int(a)
        return str(a)
    else:
        #return ord(a) - 55
        return chr(int(a) + 55)

result = np.zeros([N_test - N + 1, 2], dtype = object)

result[0, 0] = "ID"
result[0, 1] = "Class"

for i in range(N_test - N):
    img_id = i + N + 1
    #print("img_id", img_id)
    result[i + 1, 0] = str(img_id)
    result[i + 1, 1] = conv_int_to_lbl(np.argmax(out_test_numpy[i]))
    #np.argmax(out_test_numpy[i])

result

np.savetxt("result.csv", result, delimiter=",", fmt="%s")

print("end")

end
